# 1. <a id='toc1_'></a>[Clustering](#toc0_)

**Table of contents**<a id='toc0_'></a>    
1. [Clustering](#toc1_)    
1.1. [Import dependencies](#toc1_1_)    
1.2. [Load the Train datasets](#toc1_2_)    
1.2.1. [Load the scaled-extended data](#toc1_2_1_)    
1.2.2. [Load the nomalized-extended data: The data used for the AEs](#toc1_2_2_)    
1.2.3. [Load the data without scaling](#toc1_2_3_)    
1.2.4. [Summary](#toc1_2_4_)    
1.3. [Get the indecies from ``train_df`` saved for follow-up](#toc1_3_)    
1.4. [Load the trained cluster models](#toc1_4_)    
1.4.1. [Load the ``DAE_gmm_20_features_7_clusters_model``](#toc1_4_1_)    
1.4.2. [Load the ``gmm_major_features_7_clusters_model``](#toc1_4_2_)    
1.4.3. [Load the ``gmm_selected_best_20_features_7_clusters_model``](#toc1_4_3_)    
1.5. [Cluster](#toc1_5_)    
1.5.1. [Cluster a 100e3 samples from ``encoded_features_transformer_seq`` data using the ``DAE_gmm_20_features_7_clusters_model`` model](#toc1_5_1_)    
1.5.2. [Cluster a 100e3 samples from ``scaled_train_df`` data using the ``gmm_major_features_7_clusters_model`` model](#toc1_5_2_)    
1.5.3. [Cluster a 100e3 samples from ``scaled_train_df`` data using the ``gmm_selected_best_20_features_7_clusters_model`` model](#toc1_5_3_)    
1.6. [Pad the cluster labels to the sampled ``train_df``](#toc1_6_)    
1.7. [Behaviour Analysisi](#toc1_7_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1.1. <a id='toc1_1_'></a>[Import dependencies](#toc1_1_)

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
##> import libraries
import sys
from pathlib import Path
# Add root directory to path for imports >
root_dir = Path.cwd().resolve().parent
if root_dir.exists():
    sys.path.append(str(root_dir))
else:
    raise FileNotFoundError('Root directory not found')

from pprint import pprint

import pickle
import numpy as np
import torch
import shap
import pandas as pd
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

#> import custom libraries
from src.cluster import cluster_and_visualize, plot_bic_aic, flatten_trajectory_data, SHAPAnalysis, sample_features_and_indices, cluster, plot_cluster, describe_clusters
from src.models import DenoiseAutoEncoder, TransformerDenoiseAutoEncoder
from src.load import load_datasets, load_df_to_dataset
from src.plot import plot_mean_std_feature, plot_cluster_counts
from src.traj_dataloader import DenoiseAutoencoderSequencedDataset
from src.train import train_and_evaluate
from src.load import stratified_sample_df as ss_sample

import joblib
from collections import defaultdict

#> Plot
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots  # https://github.com/garrettj403/SciencePlots?tab=readme-ov-file
plt.style.use(['science', 'grid', 'notebook'])  # , 'ieee'

# %matplotlib inline
%matplotlib widget

In [4]:
data_dir = root_dir / 'data'
data_dir = data_dir.resolve()
if not data_dir.exists():
    raise FileNotFoundError('Data directory not found')

# assets_dir = data_dir / 'local' / 'aistraj' / 'tvt_assets'
assets_dir = root_dir.parents[2] / 'aistraj' / 'tvt_assets'
assets_dir = assets_dir.resolve()
if not assets_dir.exists():
    raise FileNotFoundError('Assets directory not found')

models_dir = root_dir / 'models'
models_dir = models_dir.resolve()
if not models_dir.exists():
    raise FileNotFoundError('Models directory not found')

scaled_tvt_data_import_assets_dir = assets_dir / 'scaled' 
scaled_tvt_data_import_assets_dir = scaled_tvt_data_import_assets_dir.resolve()
if not scaled_tvt_data_import_assets_dir.exists():
    raise FileNotFoundError('Train-Validate-Test Pickled Data directory not found')

extend_tvt_data_import_assets_dir = assets_dir / 'extended' 
extend_tvt_data_import_assets_dir = extend_tvt_data_import_assets_dir.resolve()
if not extend_tvt_data_import_assets_dir.exists():
    raise FileNotFoundError('Train-Validate-Test Pickled Data directory not found')

tvt_data_import_assets_dir = assets_dir / 'original' 
tvt_data_import_assets_dir = tvt_data_import_assets_dir.resolve()
if not tvt_data_import_assets_dir.exists():
    raise FileNotFoundError('Train-Validate-Test Pickled Data directory not found')

# Images dir
images_dir = root_dir / 'images_and_description'
images_dir = images_dir.resolve()
if not images_dir.exists():
    raise FileNotFoundError('images directory not found')

## 1.2. <a id='toc1_2_'></a>[Load the Train datasets](#toc1_2_)

Laveraging that the index is preserved during the training with different features sets. There is no need to descale rather to decode the data. Just follow the index in the original datasets.

### 1.2.1. <a id='toc1_2_1_'></a>[Load the scaled-extended data](#toc1_2_1_)

In [4]:
# Define the paths to the tvt files
train_pickle_path = scaled_tvt_data_import_assets_dir / 'scaled_cleaned_extended_train_df.parquet'

scaled_train_df = load_df_to_dataset(train_pickle_path).data

### 1.2.2. <a id='toc1_2_2_'></a>[Load the nomalized-extended data: The data used for the AEs](#toc1_2_2_)

In [7]:
## Load all the datasteps (input, latent, decoded) of the Denoising AE >>
pickle_save_path=Path(models_dir / 'sga')
model_name = 'transformer_denoiseautoencoder_model_parquet'
pickle_dir = pickle_save_path / Path(model_name).stem
with open(pickle_dir / "encoded_features_train.pkl", "rb") as f:
    encoded_features_transformer_seq = pickle.load(f)
with open(pickle_dir / "all_inputs_train.pkl", "rb") as f:
    all_inputs_transformer_seq = pickle.load(f)
with open(pickle_dir / "all_reconstructions_train.pkl", "rb") as f:
    all_reconstructions_transformer_seq = pickle.load(f)

In [8]:
## Print the shapes of the data >>
print (encoded_features_transformer_seq.shape)
print (all_inputs_transformer_seq.shape)
print (all_reconstructions_transformer_seq.shape)

(57444, 256, 4)
(57444, 256, 20)
(57444, 256, 20)


In [9]:
## Flatten the data in 2D (as a table) >>
encoded_features_transformer_seq = flatten_trajectory_data(encoded_features_transformer_seq, flatten_mode='fine_grained_behavior')
all_inputs_transformer_seq = flatten_trajectory_data(all_inputs_transformer_seq, flatten_mode='fine_grained_behavior')
all_reconstructions_transformer_seq = flatten_trajectory_data(all_reconstructions_transformer_seq, flatten_mode='fine_grained_behavior')

In [10]:
## Print the shapes of the data >>
print (encoded_features_transformer_seq.shape)
print (all_inputs_transformer_seq.shape)
print (all_reconstructions_transformer_seq.shape)

(14705664, 4)
(14705664, 20)
(14705664, 20)


Now we have all the datasets needed for the AE in the following sequence:
1. ``all_inputs_transformer_seq`` : The (min-max normalised) input data for the Denoising AE (DAE) 
2. ``encoded_features_transformer_seq`` : The latent space of the DAE
3. ``all_reconstructions_transformer_seq``: The reconstructued (output) data from the DAE

### 1.2.3. <a id='toc1_2_3_'></a>[Load the data without scaling](#toc1_2_3_)

In [11]:
# Define the paths to the tvt (non-scaled - original) data files: Only the train is needed
train_pickle_path = extend_tvt_data_import_assets_dir / 'cleaned_extended_train_df.parquet'
# Load
train_df = load_df_to_dataset(train_pickle_path).data

### 1.2.4. <a id='toc1_2_4_'></a>[Summary](#toc1_2_4_)

+ **Scaled Train dataset**: ``scaled_train_df``
+ **AE-related Train dataset**
  1. ``all_inputs_transformer_seq`` : The (min-max normalised) input data for the Denoising AE (DAE) 
  2. ``encoded_features_transformer_seq`` : The latent space of the DAE
  3. ``all_reconstructions_transformer_seq``: The reconstructued (output) data from the DAE
+ **Train dataset without scaling**: ``train_df``

## 1.3. <a id='toc1_3_'></a>[Get the indecies from ``train_df`` saved for follow-up](#toc1_3_)

In [12]:
# Reset the index of the tain_df and move the original index to a new column called 'original_index'
train_df.reset_index(inplace=True)
train_df.rename(columns={'index': 'original_index'}, inplace=True)

In [13]:
train_df

,original_index,epoch,stopped,cog_c,aad,rot_c,speed_c,distance_c,acc_c,cdd,...,lon,lat,obj_id,datetime,season,part_of_day,month_sin,month_cos,hour_sin,hour_cos
0,11605,1657070180,0,103.223207,1.914476,0.191448,1.305838,13.058376,0.007239,372.140836,...,10.14568,54.365526,255806357,2022-07-06 01:16:20,1,2,-0.5,-0.866025,0.258819,0.965926
1,11606,1657070190,0,104.746263,1.523056,0.152306,1.368473,13.68473,0.006264,385.825566,...,10.145883,54.365495,255806357,2022-07-06 01:16:30,1,2,-0.5,-0.866025,0.258819,0.965926
2,11607,1657070200,0,104.203357,0.542906,-0.054291,1.458833,14.588328,0.009036,400.413894,...,10.146101,54.365463,255806357,2022-07-06 01:16:40,1,2,-0.5,-0.866025,0.258819,0.965926
3,11608,1657070210,0,103.046189,1.157167,-0.115717,1.595331,15.953308,0.01365,416.367201,...,10.14634,54.36543,255806357,2022-07-06 01:16:50,1,2,-0.5,-0.866025,0.258819,0.965926
4,11609,1657070220,0,103.510316,0.464127,0.046413,1.679767,16.797667,0.008444,433.164869,...,10.146591,54.365395,255806357,2022-07-06 01:17:00,1,2,-0.5,-0.866025,0.258819,0.965926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14705495,118315304,1683331160,0,199.900217,0.262915,0.026292,4.086556,40.865558,0.044867,4163.814006,...,10.208204,54.414996,210524000,2023-05-05 23:59:20,0,2,0.5,-0.866025,-0.258819,0.965926
14705496,118315305,1683331170,0,198.168437,1.731781,-0.173178,4.23632,42.363198,0.014976,4206.177204,...,10.208,54.414634,210524000,2023-05-05 23:59:30,0,2,0.5,-0.866025,-0.258819,0.965926
14705497,118315306,1683331180,0,196.815691,1.352745,-0.135275,4.466943,44.669433,0.023062,4250.846637,...,10.207801,54.41425,210524000,2023-05-05 23:59:40,0,2,0.5,-0.866025,-0.258819,0.965926
14705498,118315307,1683331190,0,197.284832,0.469141,0.046914,4.434698,44.346978,-0.003225,4295.193615,...,10.207597,54.41387,210524000,2023-05-05 23:59:50,0,2,0.5,-0.866025,-0.258819,0.965926


## 1.4. <a id='toc1_4_'></a>[Load the trained cluster models](#toc1_4_)

Here we have three models trained with the train data but with different _scaling_ sets. The models are:
1. ``DAE_gmm_20_features_7_clusters_model``: GMM trained model on the latent space data (``encoded_features_transformer_seq``)
2. ``gmm_major_features_7_clusters_model``: GMM trained model on the major features data (``scaled_train_df``)
3. ``gmm_selected_best_10_features_7_clusters_model``: GMM trained model on the selected best features data (``scaled_train_df``)

### 1.4.1. <a id='toc1_4_1_'></a>[Load the ``DAE_gmm_20_features_7_clusters_model``](#toc1_4_1_)

In [14]:
# Directory to the saved model >
gmm_dae_model_dir = models_dir / 'sga' /  'DAE_gmm_20_features_7_clusters_model.joblib'

# Load the model >
gmm_dae_model = joblib.load(gmm_dae_model_dir)

gmm_dae_model

GaussianMixture(n_components=7, n_init=10, random_state=0)

### 1.4.2. <a id='toc1_4_2_'></a>[Load the ``gmm_major_features_7_clusters_model``](#toc1_4_2_)

In [15]:
# Directory to the saved model >
gmm_major_features_model_dir = models_dir / 'gaf' /  'gmm_major_features_7_clusters_model.joblib'

# Load the model >
gmm_major_features_model = joblib.load(gmm_major_features_model_dir)

gmm_major_features_model

GaussianMixture(n_components=7, n_init=10, random_state=0)

### 1.4.3. <a id='toc1_4_3_'></a>[Load the ``gmm_selected_best_10_features_7_clusters_model``](#toc1_4_3_)

In [16]:
# Directory to the saved model >
gmm_selected_features_model_dir = models_dir / 'gaf' /  'gmm_selected_best_10_features_7_clusters_model.joblib'

# Load the model >
gmm_selected_features_model = joblib.load(gmm_selected_features_model_dir)

gmm_selected_features_model

GaussianMixture(n_components=7, n_init=10, random_state=0)

## 1.5. <a id='toc1_5_'></a>[Cluster](#toc1_5_)

### 1.5.1. <a id='toc1_5_1_'></a>[Cluster a 100e3 samples from ``encoded_features_transformer_seq`` data using the ``DAE_gmm_20_features_7_clusters_model`` model](#toc1_5_1_)

+ Sample from the ``encoded_features_transformer_seq`` data

In [17]:
sampled_features, sampled_indices = sample_features_and_indices (encoded_features_transformer_seq[:14705500], 
                                                                 train_df.index[:14705500], 
                                                                 100000)

+ Predict and get scores

In [18]:
# GMM cluster
model_name = 'DAE_gmm_20_features_7_clusters_final'
cluster_method = 'gmm'  # 'kmeans', 'gmm', 'hdbscan
n_clusters = 7
dae_df, dae_scores, _ = cluster(pd.DataFrame(sampled_features), 
                                n_clusters=n_clusters,
                                cluster_method=cluster_method, 
                                distance_metric='euclidean', 
                                append_to_df=True, 
                                save_model_path=models_dir,
                                save_model_name=model_name,
                                verbose=2)

The average silhouette_score is : 0.8497963871423618
The Calinski-Harabasz score is : 872078.3462158148
The Davies-Bouldin score is : 0.2726383688142084
Saving the trained model to  /data1/gfalouji/repos/behaviour_analysis/sv-nba-analysis/models/DAE_gmm_20_features_7_clusters_final_model.joblib
completed with success!


### 1.5.2. <a id='toc1_5_2_'></a>[Cluster a 100e3 samples from ``scaled_train_df`` data using the ``gmm_major_features_7_clusters_model`` model](#toc1_5_2_)

+ From the ``scaled_train_df`` data, select the rows in ``sampled_indices``

In [19]:
major_features = ['season', 'part_of_day',
                  'distance_c', 'dist_ww', 'dist_ra', 'dist_cl', 'dist_ma', 
                  'cog_c', 'rot_c', 'speed_c', 'acc_c', 'lon', 'lat']

In [20]:
major_feat_df = scaled_train_df.iloc[sampled_indices,:][major_features]
major_feat_df

,season,part_of_day,distance_c,dist_ww,dist_ra,dist_cl,dist_ma,cog_c,rot_c,speed_c,acc_c,lon,lat
11177025,0,0,9.773429e-06,0.006345,0.002979,0.006413,0.002979,0.534125,-0.002248,0.156016,-0.981735,0.170650,0.068030
10922261,0,1,4.791735e-06,0.003453,0.003802,0.004347,0.003802,0.692489,0.000299,-0.257508,-0.212930,0.172225,0.071417
2798670,1,2,5.651457e-06,0.005522,0.002205,0.005845,0.002205,0.906041,-0.001545,-0.186144,-0.446655,0.170578,0.069383
3829547,1,2,4.464625e-08,0.005596,0.002296,0.005939,0.002296,0.896979,0.001407,-0.651557,-0.032515,0.170447,0.069419
10556381,0,0,1.654757e-05,0.003813,0.002729,0.004135,0.002729,0.490989,-0.000701,0.718328,-0.404028,0.172316,0.070213
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747068,1,0,1.511057e-05,0.005956,0.002594,0.006027,0.002594,0.607893,0.000697,0.599045,-0.243445,0.170985,0.068290
2255389,1,1,1.690702e-05,0.003626,0.002860,0.003920,0.002860,0.043911,0.000100,0.748165,0.195251,0.172567,0.070240
9643563,0,1,1.543341e-05,0.005585,0.002207,0.005764,0.002207,0.249087,0.000588,0.625843,2.945784,0.170955,0.068861
366011,1,0,7.408287e-06,0.006722,0.003354,0.006784,0.003354,0.129738,0.000287,-0.040311,-1.312124,0.170337,0.067769


+ Predict and get scores

In [21]:
# GMM cluster
model_name = 'gmm_major_features_7_clusters_final'
cluster_method = 'gmm'  # 'kmeans', 'gmm', 'hdbscan
n_clusters = 7
major_df, major_scores, _ = cluster(major_feat_df, 
                                    n_clusters=n_clusters,
                                    cluster_method=cluster_method, 
                                    distance_metric='euclidean', 
                                    append_to_df=True, 
                                    save_model_path=models_dir,
                                    save_model_name=model_name,
                                    verbose=2)

The average silhouette_score is : 0.4121503684743852
The Calinski-Harabasz score is : 17490.846924344078
The Davies-Bouldin score is : 5.265608735580207
Saving the trained model to  /data1/gfalouji/repos/behaviour_analysis/sv-nba-analysis/models/gmm_major_features_7_clusters_final_model.joblib
completed with success!


### 1.5.3. <a id='toc1_5_3_'></a>[Cluster a 100e3 samples from ``scaled_train_df`` data using the ``gmm_selected_best_10_features_7_clusters_model`` model](#toc1_5_3_)

+ From the ``scaled_train_df`` data, select the targeted columns

In [22]:
## Load selected features average scores >>
selected_features_path = models_dir / 'selected_features' / 'all_selected_features.csv'
all_features_scores = pd.read_csv(selected_features_path)

display(all_features_scores)

,selected_features,feature_importance_pca,feature_importance_vtm,feature_importance_mlp,mean,variance
0,acc_c,1.032544e-13,1.000000e+00,0.794431,5.981436e-01,2.788965e-01
1,month_sin,1.000000e+00,6.784170e-07,0.585267,5.284224e-01,2.524231e-01
2,distance_c,1.724464e-07,0.000000e+00,1.000000,3.333334e-01,3.333333e-01
3,dist_ra,9.316073e-12,1.205393e-12,0.992225,3.307418e-01,3.281705e-01
4,lat,0.000000e+00,1.879845e-12,0.987764,3.292547e-01,3.252259e-01
5,dist_ww,2.112695e-10,1.680394e-12,0.972620,3.242066e-01,3.153297e-01
6,dist_cl,3.995835e-12,1.156549e-12,0.971416,3.238054e-01,3.145498e-01
7,lon,0.000000e+00,1.845572e-12,0.967629,3.225430e-01,3.121019e-01
8,dist_ma,7.556275e-13,1.205393e-12,0.960541,3.201802e-01,3.075462e-01
9,rot_c,1.990598e-07,1.429430e-10,0.954455,3.181518e-01,3.036615e-01


In [23]:
best_features_stop_indx = 10  # Take the first 10 of the sorted features
## Create a list of the selected features
selected_features = all_features_scores['selected_features'].values[:best_features_stop_indx]
print(f'The selected features are: {selected_features}')

The selected features are: ['acc_c' 'month_sin' 'distance_c' 'dist_ra' 'lat' 'dist_ww' 'dist_cl'
 'lon' 'dist_ma' 'rot_c']


In [24]:
selected_df= scaled_train_df.iloc[sampled_indices,:][selected_features]
selected_df

,acc_c,month_sin,distance_c,dist_ra,lat,dist_ww,dist_cl,lon,dist_ma,rot_c
11177025,-0.981735,8.660254e-01,9.773429e-06,0.002979,0.068030,0.006345,0.006413,0.170650,0.002979,-0.002248
10922261,-0.212930,5.000000e-01,4.791735e-06,0.003802,0.071417,0.003453,0.004347,0.172225,0.003802,0.000299
2798670,-0.446655,-5.000000e-01,5.651457e-06,0.002205,0.069383,0.005522,0.005845,0.170578,0.002205,-0.001545
3829547,-0.032515,-5.000000e-01,4.464625e-08,0.002296,0.069419,0.005596,0.005939,0.170447,0.002296,0.001407
10556381,-0.404028,1.224647e-16,1.654757e-05,0.002729,0.070213,0.003813,0.004135,0.172316,0.002729,-0.000701
...,...,...,...,...,...,...,...,...,...,...
3747068,-0.243445,-5.000000e-01,1.511057e-05,0.002594,0.068290,0.005956,0.006027,0.170985,0.002594,0.000697
2255389,0.195251,-8.660254e-01,1.690702e-05,0.002860,0.070240,0.003626,0.003920,0.172567,0.002860,0.000100
9643563,2.945784,1.224647e-16,1.543341e-05,0.002207,0.068861,0.005585,0.005764,0.170955,0.002207,0.000588
366011,-1.312124,1.224647e-16,7.408287e-06,0.003354,0.067769,0.006722,0.006784,0.170337,0.003354,0.000287


+ Predict and get scores

In [25]:
# GMM cluster
# GMM cluster
model_name = 'gmm_selected_best_10_features_7_clusters_model'
cluster_method = 'gmm'  # 'kmeans', 'gmm', 'hdbscan
n_clusters = 7
select_df, select_scores, _ = cluster(selected_df, 
                                      n_clusters=n_clusters,
                                      cluster_method=cluster_method, 
                                      distance_metric='euclidean', 
                                      append_to_df=True, 
                                      save_model_path=models_dir,
                                      save_model_name=model_name,
                                      verbose=2)

The average silhouette_score is : 0.4452951058288049
The Calinski-Harabasz score is : 36758.95156277374
The Davies-Bouldin score is : 2.6907939317885843
Saving the trained model to  /data1/gfalouji/repos/behaviour_analysis/sv-nba-analysis/models/gmm_selected_best_10_features_7_clusters_model_model.joblib
completed with success!


## 1.6. <a id='toc1_6_'></a>[Pad the cluster labels to the sampled ``train_df``](#toc1_6_)

+ Initialise the final dataframe with cluster labels

In [26]:
clustered_df = train_df.iloc[sampled_indices,:].copy()

+ Add to ``cluster_df`` the cluster labels from ``dae_df``

In [27]:
clustered_df['dae_gmm_clusters'] =dae_df['gmm_clusters'].values

+ Add to ``cluster_df`` the cluster labels from ``major_df``

In [30]:
clustered_df['major_gmm_clusters'] = major_df['gmm_clusters'].values

+ Add to ``cluster_df`` the cluster labels from ``major_df``

In [31]:
clustered_df['selected_gmm_clusters'] = selected_df['gmm_clusters'].values

+ Display the final ``clustered_df``

In [32]:
clustered_df

,original_index,epoch,stopped,cog_c,aad,rot_c,speed_c,distance_c,acc_c,cdd,...,datetime,season,part_of_day,month_sin,month_cos,hour_sin,hour_cos,dae_gmm_clusters,major_gmm_clusters,selected_gmm_clusters
11177025,88643178,1651062360,0,192.284979,8.092272,-0.809227,3.080386,30.803857,-0.014123,14867.092647,...,2022-04-27 12:26:00,0,0,8.660254e-01,-0.500000,1.224647e-16,-1.000000e+00,3,4,3
10922261,86165953,1651414800,0,249.296089,1.075787,0.107579,1.510257,15.102573,-0.00315,4200.399894,...,2022-05-01 14:20:00,0,1,5.000000e-01,-0.866025,-5.000000e-01,-8.660254e-01,1,4,0
2798670,22356903,1657307160,0,326.17491,5.561214,-0.556121,1.781224,17.81224,-0.006486,943.410321,...,2022-07-08 19:06:00,1,2,-5.000000e-01,-0.866025,-9.659258e-01,2.588190e-01,2,0,0
3829547,29843047,1657485750,0,322.912473,5.066277,0.506628,0.014072,0.140716,-0.000575,11299.647552,...,2022-07-10 20:42:30,1,2,-5.000000e-01,-0.866025,-8.660254e-01,5.000000e-01,2,0,0
10556381,83566900,1654516340,0,176.756161,2.522277,-0.252228,5.215458,52.154577,-0.005877,47245.947717,...,2022-06-06 11:52:20,0,0,1.224647e-16,-1.000000,2.588190e-01,-9.659258e-01,6,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747068,29010285,1657454320,0,218.841638,2.508311,0.250831,4.762545,47.625448,-0.003585,36847.261715,...,2022-07-10 11:58:40,1,0,-5.000000e-01,-0.866025,2.588190e-01,-9.659258e-01,6,0,0
2255389,17677960,1659551120,0,15.808122,0.360303,0.03603,5.328749,53.287488,0.002676,7209.94456,...,2022-08-03 18:25:20,1,1,-8.660254e-01,-0.500000,-1.000000e+00,-1.836970e-16,5,0,0
9643563,75470836,1685898790,0,89.671164,2.116945,0.211695,4.864297,48.642969,0.041935,1173.095206,...,2023-06-04 17:13:10,0,1,1.224647e-16,-1.000000,-9.659258e-01,-2.588190e-01,1,4,3
366011,3058763,1687423930,0,46.70566,1.032495,0.103249,2.334941,23.349413,-0.018839,7685.923656,...,2023-06-22 08:52:10,1,0,1.224647e-16,-1.000000,8.660254e-01,-5.000000e-01,0,0,3


+ Save the final ``clustered_df``

In [33]:
target_save_path = data_dir / 'clusters' / 'final_clusters_df.csv'
clustered_df.to_csv(target_save_path, index=False)

## 1.7. <a id='toc1_7_'></a>[Behaviour Analysis](#toc1_7_)

### Import 'final_clusters_df.csv'

In [6]:
target_save_path = data_dir / 'clusters' / 'final_clusters_df.csv'

clustered_df = pd.read_csv (target_save_path)

### Definite the different feature list
+ 'group_columns': List of column names to group by.
+ 'describe_features': List of features to describe in each group.
+ 'selected_features': List of features which represent the temporal and spatial patterns

In [13]:
group_columns = ['dae_gmm_clusters', 'major_gmm_clusters', 'selected_gmm_clusters']

describe_features = [
    'season', 'part_of_day',
    'aad', 'cdd', 'dir_ccs', 'cog_c', 'rot_c', 'distance_c', 'dist_ww', 'dist_ra',
    'dist_cl', 'dist_ma', 'speed_c', 'acc_c', 'lon', 'lat'
]

selected_features = ['part_of_day', 'season', 'cdd', 'distance_c']

### Get new dataframe with describe_features and cluster labels with specified method
+ used for plots

In [10]:
dae_cluster_df = clustered_df [describe_features+[group_columns[0]]]
major_cluster_df = clustered_df [describe_features+[group_columns[1]]]
selected_cluster_df = clustered_df [describe_features+[group_columns[2]]]

### Get describe dataframe for different methods

In [14]:
dae_describe_df = describe_clusters (clustered_df, group_columns[0], selected_features)

major_describe_df = describe_clusters (clustered_df, group_columns[1], selected_features)

selected_describe_df = describe_clusters (clustered_df, group_columns[2], selected_features)

In [15]:
dae_describe_df

dae_gmm_clusters              0             1             2             3  \
part_of_day count  1.424300e+04  1.830000e+04  1.139000e+04  1.869600e+04   
            mean   1.685038e-03  9.955738e-01  1.939596e+00  1.668806e-02   
            std    5.680708e-02  6.801081e-02  2.382440e-01  1.281033e-01   
            min    0.000000e+00  0.000000e+00  1.000000e+00  0.000000e+00   
            25%    0.000000e+00  1.000000e+00  2.000000e+00  0.000000e+00   
            50%    0.000000e+00  1.000000e+00  2.000000e+00  0.000000e+00   
            75%    0.000000e+00  1.000000e+00  2.000000e+00  0.000000e+00   
            max    2.000000e+00  2.000000e+00  2.000000e+00  1.000000e+00   
season      count  1.424300e+04  1.830000e+04  1.139000e+04  1.869600e+04   
            mean   7.088394e-01  7.437705e-01  9.191396e-01  7.253958e-01   
            std    9.537676e-01  9.172417e-01  1.067834e+00  9.152999e-01   
            min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            25%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            50%    0.000000e+00  0.000000e+00  1.000000e+00  0.000000e+00   
            75%    1.000000e+00  1.000000e+00  2.000000e+00  1.000000e+00   
            max    3.000000e+00  3.000000e+00  3.000000e+00  3.000000e+00   
cdd         count  1.424300e+04  1.830000e+04  1.139000e+04  1.869600e+04   
            mean   1.065027e+04  2.208502e+04  1.980688e+04  1.630941e+04   
            std    5.940181e+04  1.069478e+05  1.853669e+05  9.329666e+04   
            min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            25%    2.127916e+03  2.886480e+03  1.860412e+03  2.899660e+03   
            50%    6.103254e+03  8.126382e+03  6.991691e+03  7.486013e+03   
            75%    1.118281e+04  1.515570e+04  1.167126e+04  1.297989e+04   
            max    2.405233e+06  5.031462e+06  7.258723e+06  3.550568e+06   
distance_c  count  1.424300e+04  1.830000e+04  1.139000e+04  1.869600e+04   
            mean   2.730513e+01  2.796840e+01  2.731789e+01  2.654227e+01   
            std    7.871186e+01  7.398508e+01  4.038727e+01  8.578964e+01   
            min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            25%    5.603022e+00  8.151952e+00  2.028675e+00  8.529565e+00   
            50%    2.471184e+01  2.558322e+01  2.219445e+01  2.424072e+01   
            75%    4.253799e+01  4.204943e+01  4.772553e+01  3.813650e+01   
            max    7.822187e+03  6.791703e+03  2.584989e+03  8.036960e+03   

dae_gmm_clusters              4             5             6  
part_of_day count  1.224400e+04  1.279500e+04  1.233200e+04  
            mean   1.999673e+00  1.017663e+00  2.432695e-04  
            std    2.213525e-02  1.317290e-01  1.559584e-02  
            min    0.000000e+00  1.000000e+00  0.000000e+00  
            25%    2.000000e+00  1.000000e+00  0.000000e+00  
            50%    2.000000e+00  1.000000e+00  0.000000e+00  
            75%    2.000000e+00  1.000000e+00  0.000000e+00  
            max    2.000000e+00  2.000000e+00  1.000000e+00  
season      count  1.224400e+04  1.279500e+04  1.233200e+04  
            mean   8.405750e-01  7.092614e-01  7.326468e-01  
            std    1.048789e+00  9.117051e-01  9.481547e-01  
            min    0.000000e+00  0.000000e+00  0.000000e+00  
            25%    0.000000e+00  0.000000e+00  0.000000e+00  
            50%    0.000000e+00  0.000000e+00  0.000000e+00  
            75%    2.000000e+00  1.000000e+00  1.000000e+00  
            max    3.000000e+00  3.000000e+00  3.000000e+00  
cdd         count  1.224400e+04  1.279500e+04  1.233200e+04  
            mean   9.652415e+03  2.205790e+04  1.171498e+04  
            std    8.304307e+04  1.048507e+05  6.133818e+04  
            min    0.000000e+00  0.000000e+00  0.000000e+00  
            25%    1.384654e+03  2.947592e+03  2.324827e+03  
            50%    5.450250e+03  8.178068e+03  6.329549e+03  
            75%    1.042644e+04 

In [16]:
major_describe_df

major_gmm_clusters             0             1             2             3  \
part_of_day count   4.074800e+04  7.600000e+02  6.000000e+00  3.000000e+00   
            mean    7.940512e-01  1.098684e+00  6.666667e-01  3.333333e-01   
            std     7.962424e-01  8.400379e-01  5.163978e-01  5.773503e-01   
            min     0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            25%     0.000000e+00  0.000000e+00  2.500000e-01  0.000000e+00   
            50%     1.000000e+00  1.000000e+00  1.000000e+00  0.000000e+00   
            75%     1.000000e+00  2.000000e+00  1.000000e+00  5.000000e-01   
            max     2.000000e+00  2.000000e+00  1.000000e+00  1.000000e+00   
season      count   4.074800e+04  7.600000e+02  6.000000e+00  3.000000e+00   
            mean    1.587489e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            std     7.628305e-01  0.000000e+00  0.000000e+00  0.000000e+00   
            min     1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            25%     1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            50%     1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            75%     2.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            max     3.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
cdd         count   4.074800e+04  7.600000e+02  6.000000e+00  3.000000e+00   
            mean    1.369088e+04  3.194356e+05  1.547561e+06  1.723469e+06   
            std     7.480605e+04  8.716462e+05  1.344138e+06  6.898145e+05   
            min     0.000000e+00  0.000000e+00  1.221489e+05  1.234263e+06   
            25%     2.683311e+03  3.171194e+03  4.547362e+05  1.328977e+06   
            50%     7.090813e+03  8.165049e+03  1.330935e+06  1.423690e+06   
            75%     1.191800e+04  4.305165e+04  2.647475e+06  1.968072e+06   
            max     3.209768e+06  7.258723e+06  3.252356e+06  2.512454e+06   
distance_c  count   4.074800e+04  7.600000e+02  6.000000e+00  3.000000e+00   
            mean    2.901191e+01  1.880095e+02  2.054898e+03  6.990756e+03   
            std     2.006621e+01  5.985388e+02  1.461793e+03  7.519313e+02   
            min     0.000000e+00  0.000000e+00  4.172740e+01  6.358379e+03   
            25%     1.269178e+01  3.511592e+01  1.467281e+03  6.575041e+03   
            50%     2.782027e+01  7.883623e+01  1.991347e+03  6.791703e+03   
            75%     4.545707e+01  1.131392e+02  2.395637e+03  7.306945e+03   
            max     1.157823e+02  8.036960e+03  4.481973e+03  7.822187e+03   

major_gmm_clusters             4             5             6  
part_of_day count   5.187300e+04  6.609000e+03      1.000000  
            mean    7.535134e-01  8.881828e-01      2.000000  
            std     7.893544e-01  8.292809e-01           NaN  
            min     0.000000e+00  0.000000e+00      2.000000  
            25%     0.000000e+00  0.000000e+00      2.000000  
            50%     1.000000e+00  1.000000e+00      2.000000  
            75%     1.000000e+00  2.000000e+00      2.000000  
            max     2.000000e+00  2.000000e+00      2.000000  
season      count   5.187300e+04  6.609000e+03      1.000000  
            mean    0.000000e+00  1.732486e+00      3.000000  
            std     0.000000e+00  8.147991e-01           NaN  
            min     0.000000e+00  0.000000e+00      3.000000  
            25%     0.000000e+00  1.000000e+00      3.000000  
            50%     0.000000e+00  2.000000e+00      3.000000  
            75%     0.000000e+00  2.000000e+00      3.000000  
            max     0.000000e+00  3.000000e+00      3.000000  
cdd         count   5.187300e+04  6.609000e+03      1.000000  
            mean    1.393305e+04  1.411976e+04  83410.526665  
            std     5.637022e+04  4.728893e+04           NaN  
            min     0.000000e+00  7.383993e-02  83410.526665  
            25%     2.345031e+03  4.719461e+02  83410.526665  
            50%     6.854138e+03  6.066

In [17]:
selected_describe_df

selected_gmm_clusters             0             1             2             3  \
part_of_day count      4.970000e+04  8.000000e+00  3.000000e+00  3.412100e+04   
            mean       7.378471e-01  6.250000e-01  3.333333e-01  8.021453e-01   
            std        7.799811e-01  9.161254e-01  5.773503e-01  8.005948e-01   
            min        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            25%        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            50%        1.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00   
            75%        1.000000e+00  1.250000e+00  5.000000e-01  1.000000e+00   
            max        2.000000e+00  2.000000e+00  1.000000e+00  2.000000e+00   
season      count      4.970000e+04  8.000000e+00  3.000000e+00  3.412100e+04   
            mean       7.317505e-01  0.000000e+00  0.000000e+00  7.731309e-01   
            std        9.269556e-01  0.000000e+00  0.000000e+00  9.793711e-01   
            min        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            25%        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            50%        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
            75%        1.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00   
            max        3.000000e+00  0.000000e+00  0.000000e+00  3.000000e+00   
cdd         count      4.970000e+04  8.000000e+00  3.000000e+00  3.412100e+04   
            mean       1.186919e+04  6.447826e+05  1.723469e+06  1.597164e+04   
            std        6.762352e+04  6.699686e+05  6.898145e+05  6.066440e+04   
            min        0.000000e+00  4.255559e+04  1.234263e+06  3.592880e+00   
            25%        2.347081e+03  2.990411e+05  1.328977e+06  3.226239e+03   
            50%        6.546936e+03  5.090624e+05  1.423690e+06  7.461511e+03   
            75%        1.161694e+04  7.020936e+05  1.968072e+06  1.246686e+04   
            max        5.031462e+06  2.177122e+06  2.512454e+06  3.209768e+06   
distance_c  count      4.970000e+04  8.000000e+00  3.000000e+00  3.412100e+04   
            mean       2.508311e+01  5.212779e+03  6.990756e+03  3.823302e+01   
            std        1.943503e+01  1.612841e+03  7.519313e+02  1.965769e+01   
            min        0.000000e+00  3.658854e+03  6.358379e+03  0.000000e+00   
            25%        9.080379e+00  4.382496e+03  6.575041e+03  2.353804e+01   
            50%        2.404754e+01  4.574592e+03  6.791703e+03  3.950815e+01   
            75%        3.676433e+01  5.428213e+03  7.306945e+03  5.143675e+01   
            max        7.403319e+02  8.036960e+03  7.822187e+03  2.132344e+02   

selected_gmm_clusters             4             5             6  
part_of_day count      2.210000e+02  1.315100e+04  2.796000e+03  
            mean       1.108597e+00  8.867006e-01  7.875536e-01  
            std        8.774800e-01  8.321114e-01  7.961910e-01  
            min        0.000000e+00  0.000000e+00  0.000000e+00  
            25%        0.000000e+00  0.000000e+00  0.000000e+00  
            50%        1.000000e+00  1.000000e+00  1.000000e+00  
            75%        2.000000e+00  2.000000e+00  1.000000e+00  
            max        2.000000e+00  2.000000e+00  2.000000e+00  
season      count      2.210000e+02  1.315100e+04  2.796000e+03  
            mean       2.217195e-01  8.430538e-01  8.068670e-01  
            std        6.184135e-01  1.034245e+00  1.012923e+00  
            min        0.000000e+00  0.000000e+00  0.000000e+00  
            25%        0.000000e+00  0.000000e+00  0.000000e+00  
            50%        0.000000e+00  0.000000e+00  0.000000e+00  
            75%        0.000000e+00  1.000000e+00  1.000000e+00  
            max        3.000000e+00  3.000000e+00  3.000000e+00  
cdd         count      2.210000e+02  1.315100e+04  2.796000e+03  
            mean       1.009893e+06  1.554713e+04  2.089075e+04  
            std        1.386399e+06  5.311531e+04  1.188875e+05  
      

### Plots for different features with cluster labels

In [ ]:
save_path = images_dir / f'dae_part_of_day_mean_std.png'

plot_mean_std_feature(
    df=dae_cluster_df, 
    group_column='dae_gmm_clusters', 
    feature='part_of_day',
    title='Mean and Std of Part of Day across Clusters',
    xlabel='Cluster',
    ylabel='Part of Day',
    save_path=None
)

### Analyse for DAE method
+ count
+ temporal features
    - 'part_of_day'
    - 'season'
+ spatial features
    - 'cdd'
    - 'distance_c'


### Analyse for latent features method

In [ ]:
save_path = images_dir / f'dae_part_of_day_mean_std.png'

plot_mean_std_feature(
    df=dae_cluster_df, 
    group_column='dae_gmm_clusters', 
    feature='part_of_day',
    title='Mean and Std of Part of Day across Clusters',
    xlabel='Cluster',
    ylabel='Part of Day',
    save_path=save_path
)


### Analyse for major features method

In [ ]:
save_path = images_dir / f'major_part_of_day_mean_std.png'

plot_mean_std_feature(
    df=major_cluster_df, 
    group_column='major_gmm_clusters', 
    feature='part_of_day',
    title='Mean and Std of Part of Day across Clusters',
    xlabel='Cluster',
    ylabel='Part of Day',
    save_path=save_path,
    std=True
)

### Analyse for selected features method

In [ ]:
save_path = images_dir / f'selected_part_of_day_mean_std.png'

plot_mean_std_feature(
    df=selected_cluster_df, 
    group_column='selected_gmm_clusters', 
    feature='part_of_day',
    title='Mean and Std of Part of Day across Clusters',
    xlabel='Cluster',
    ylabel='Part of Day',
    save_path=save_path
)